# Task 1: Social Media Post Creation using Prompt Engineering

## Objective
This notebook demonstrates how Large Language Models (LLMs) can be used to automate **professional social media content creation** for marketing teams.

The task is implemented using **three different prompt templates**, each showcasing a **different prompting technique** used in industry.

### Scope
- Text → Text (LinkedIn post copy)
- Text → Image Prompt (creative description only)
- User-driven generic prompt template (industry-style reusable template)


## Prompting Techniques, Models, and Modalities Used

| Prompt | Purpose | Prompting Technique | Model Used | Model Modality | Model Provider (via Nexus API) |
|------|--------|--------------------|-----------|---------------|-------------------------------|
| Prompt 1 | Professional LinkedIn post generation | Role-Based + Structured + Zero-shot | gpt-4.1-nano | Text → Text | OpenAI (via Navigate Labs Nexus API) |
| Prompt 2 | Social media image generation | Zero-shot + Instruction-based + Visual Constraints | imagen-4.0-fast-generate-001 | Text → Image | Google (Imagen) via Navigate Labs Nexus API |
| Prompt 3 | Reusable industry-grade prompt template | Template-based + User-input driven (Zero-shot) | gpt-4.1-nano | Text → Text | OpenAI (via Navigate Labs Nexus API) |


::

In [ ]:
!pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI

# Ensure these are set in Colab environment
#API configuaration
os.environ["OPENAI_API_KEY"] = "YOUR API KEY"
os.environ["OPENAI_BASE_URL"] = "YOUR URL"

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL")
)


## Prompt 1: Professional LinkedIn Post Generation

### Prompt Objective
Generate a **professional, enterprise-grade LinkedIn post** announcing a product or feature.

### Prompting Technique
- Role-Based Prompting  
- Structured Prompting  
- Zero-Shot Prompting  

This is commonly used in **brand-safe content generation systems**.


In [ ]:
prompt1 = """
You are a senior B2B social media strategist working for an enterprise technology company.

Task:
Write a professional LinkedIn post announcing the launch of an AI-powered marketing analytics platform.

Guidelines:
- Professional and authoritative tone
- Focus on business value and outcomes
- Avoid technical jargon
- No emojis
- Length: 80–120 words

Output:
Only provide the LinkedIn post text.
"""

response1 = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": prompt1}]
)

print(response1.choices[0].message.content)


We are excited to announce the launch of our new AI-powered marketing analytics platform, designed to deliver deeper insights and drive measurable results. This innovative solution empowers enterprises to make more informed decisions, optimize campaign performance, and maximize ROI. By harnessing advanced data capabilities, our platform enables organizations to better understand customer behaviors and emerging market trends—unlocking opportunities for growth and competitive advantage. As the landscape evolves, our commitment is to provide tools that elevate your marketing strategies and deliver sustainable business success. Discover how our latest technology can transform your approach to data-driven marketing.


## Prompt 2: Social Media Image Generation

### Prompt Objective
Generate a **clear, professional, industry-grade marketing image** for a LinkedIn post using an image generation model.

### Prompting Technique
- Zero-Shot Prompting  
- Instruction-Based Prompting  

This approach mirrors how **creative briefs** are translated into visuals in enterprise marketing workflows.


In [ ]:
from IPython.display import Image, display
from PIL import Image as PILImage
from io import BytesIO
import base64

image_prompt = """
Professional LinkedIn marketing image announcing an AI-powered marketing analytics platform.
Clean, modern enterprise design.
Clear AI analytics dashboard with readable charts and graphs.
Business professionals analyzing data on large screens.
Blue and white corporate color palette.
Minimal text, no logos, no slogans.
High clarity, sharp focus, realistic business environment.
Flat or semi-isometric style.
Suitable for LinkedIn feed.
"""

try:
    image_response = client.images.generate(
        model="imagen-4.0-fast-generate-001",
        prompt=image_prompt,
        size="1024x1024"
    )

    image_data = image_response.data[0]

    # Case 1: URL-based response
    if hasattr(image_data, "url") and image_data.url:
        print("Image generated successfully (URL).")
        display(Image(url=image_data.url))

    # Case 2: Base64-based response
    elif hasattr(image_data, "b64_json") and image_data.b64_json:
        print("Image generated successfully (Base64).")
        img_bytes = base64.b64decode(image_data.b64_json)
        img = PILImage.open(BytesIO(img_bytes))
        display(img)

    else:
        raise ValueError("Image data not returned by the model.")

except Exception as e:
    print("Image generation failed.")
    print("Reason:", e)
    print("\nFallback: Image generation prompt")
    print(image_prompt)


Image generation failed.
Reason: Error code: 429 - {'error': {'message': 'litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {\n  "error": {\n    "code": 429,\n    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \\n* Quota exceeded for metric: generativelanguage.googleapis.com/predict_requests_per_model_per_day_paid_tier_1, limit: 70, model: imagen-4.0-fast-generate",\n    "status": "RESOURCE_EXHAUSTED",\n    "details": [\n      {\n        "@type": "type.googleapis.com/google.rpc.Help",\n        "links": [\n          {\n            "description": "Learn more about Gemini API quotas",\n            "url": "https://ai.google.dev/gemini-api/docs/rate-limits"\n          }\n        ]\n      },\n      {\n        "@type": "type.googleapis.com/google.rpc.QuotaFailure"

## Prompt 3: Reusable Industry Prompt Template

### Prompt Objective
Provide a **generic, reusable prompt template** that accepts **user input dynamically**.

### Prompting Technique
- Template-Based Prompting  
- User-Driven Prompting  
- Zero-Shot (depends on user input)

This pattern is widely used in **prompt libraries and SaaS products**.


In [ ]:
text_type = "social media caption"
user_input = """
We built a new AI tool that helps marketing teams understand their data better and make faster decisions.
"""

prompt3 = f"""
The following is text from a {text_type}.

{user_input}

Task:
Analyze the above {text_type} and provide:
1. A professional rewrite suitable for LinkedIn
2. A short image-generation prompt that visually represents the message

Guidelines:
- Professional business tone
- Clear and concise language
- Suitable for enterprise audiences

Output Format:
A. LinkedIn Post Text
B. Image Generation Prompt
"""

response3 = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=[{"role": "user", "content": prompt3}]
)

print(response3.choices[0].message.content)


A. LinkedIn Post Text  
We are pleased to announce the launch of our latest AI-powered solution designed to empower marketing teams. This innovative tool enhances data comprehension and accelerates decision-making, enabling organizations to drive more informed and timely strategic actions.

B. Image Generation Prompt  
A professional office setting with marketing professionals analyzing data charts on large screens, with sleek digital interfaces and AI symbols overlaying the scene, conveying advanced data analysis and decision-making.
